<a href="https://colab.research.google.com/github/emilziyaev/rosselkhozChallenge/blob/main/SiamMask_ez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Новый раздел

In [ ]:
%%capture
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
search_id = "1OejSwght_F4tKbaLjgmB1srBnIJDNuU-"
search = drive.CreateFile({'id': search_id})
search.GetContentFile('train.zip')

In [ ]:
!unzip ./train.zip
!rm -r ./train.zip

Archive:  ./train.zip
   creating: train/
  inflating: train/Movie_5.mkv       
  inflating: train/Movie_7.mkv       
  inflating: train/Movie_6.mkv       
  inflating: train/Movie_10.mkv      
  inflating: train/Movie_1.mkv       
  inflating: train/Movie_3.mkv       
  inflating: train/Movie_2.mkv       
  inflating: train/Movie_4.mkv       
  inflating: train/Movie_8.mkv       
  inflating: train/Movie_9.mkv       


# Track/Segment Person with SiamMask

This notebook uses an open source project [SiamMask](https://github.com/foolwood/SiamMask) to track and to segment a single person on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## Install SiamMask

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/foolwood/SiamMask.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !sed -i "/torch/d" {project_name}/requirements.txt
  !cd {project_name} && pip install -q -r requirements.txt
  !cd {project_name} && bash make.sh
  !pip install -q youtube-dl
  
import sys
sys.path.append(project_name)
sys.path.append(join(project_name, 'experiments', 'siammask_sharp'))
import time
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

import cv2
import torchvision
import cv2
import numpy as np
import torch
torch.set_grad_enabled(False)

from types import SimpleNamespace
from custom import Custom
from tools.test import *

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
     |████████████████████████████████| 13.8 MB 46.5 MB/s 
     |████████████████████████████████| 57 kB 6.9 MB/s 
     |████████████████████████████████| 12.6 MB 25.9 MB/s 
     |████████████████████████████████| 1.9 MB 51.7 MB/s 
     |████████████████████████████████| 31.2 MB 118 kB/s 
     |████████████████████████████████| 2.8 MB 57.5 MB/s 
     |████████████████████████████████| 8.8 MB 45.0 MB/s 
     |████████████████████████████████| 46 kB 136 kB/s 
     |████████████████████████████████| 129 kB 68.2 MB/s 
     |████████████████████████████████| 25.0 MB 59.3 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.15.4 which is incompatible.
xarray 0.18.2 requires numpy>=1.

In [ ]:
siammask.train().to(device)

In [ ]:
siam

## Download pretrained weights and initialize SiamMask 

In [ ]:
exp_path = join(project_name, 'experiments/siammask_sharp')
pretrained_path1 = join(exp_path, 'SiamMask_DAVIS.pth')
pretrained_path2 = join(exp_path, 'SiamMask_VOT.pth')

# download pretrained weights
if not exists(pretrained_path1):
  !cd {exp_path} && wget http://www.robots.ox.ac.uk/~qwang/SiamMask_DAVIS.pth
if not exists(pretrained_path2):
  !cd {exp_path} && wget http://www.robots.ox.ac.uk/~qwang/SiamMask_VOT.pth
    
# init SiamMask
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = load_config(SimpleNamespace(config=join(exp_path, 'config_davis.json')))
siammask = Custom(anchors=cfg['anchors'])
siammask = load_pretrain(siammask, pretrained_path1)
siammask = siammask.eval().to(device)

--2021-11-19 17:42:16--  http://www.robots.ox.ac.uk/~qwang/SiamMask_DAVIS.pth
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.robots.ox.ac.uk/~qwang/SiamMask_DAVIS.pth [following]
--2021-11-19 17:42:17--  https://www.robots.ox.ac.uk/~qwang/SiamMask_DAVIS.pth
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105878246 (101M)
Saving to: ‘SiamMask_DAVIS.pth’

SiamMask_DAVIS.pth  100%[===================>] 100.97M  27.7MB/s    in 4.0s    

2021-11-19 17:42:21 (25.3 MB/s) - ‘SiamMask_DAVIS.pth’ saved [105878246/105878246]

--2021-11-19 17:42:21--  http://www.robots.ox.ac.uk/~qwang/SiamMask_VOT.pth
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.

[2021-11-19 17:42:26,681-rk0-features.py# 66] Current training 0 layers:
	
[2021-11-19 17:42:26,683-rk0-features.py# 66] Current training 1 layers:
	
[2021-11-19 17:42:26,748-rk0-load_helper.py# 31] load pretrained model from SiamMask/experiments/siammask_sharp/SiamMask_DAVIS.pth
[2021-11-19 17:42:36,448-rk0-load_helper.py# 25] remove prefix 'module.'
[2021-11-19 17:42:36,452-rk0-load_helper.py# 18] used keys:356


# train

In [ ]:
siammask = siammask.train().to(device)

In [ ]:
from tools import train_siammask

In [ ]:
train_siammask()

# Predict

In [ ]:
f = 0
video_capture = cv2.VideoCapture()
if video_capture.open('/content/train/Movie_1.mkv'):
  width, height = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = video_capture.get(cv2.CAP_PROP_FPS)
  
  !rm -f output.mp4 output.avi
  # can't write out mp4, so try to write into an AVI file
  video_writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
  
  while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
      break

    if f == 0:
      # detect a person on the first frame with Mask R-CNN
      maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
      maskrcnn = maskrcnn.eval()
      output = maskrcnn([torchvision.transforms.functional.to_tensor((frame[...,::-1] / 255).astype(np.float32))])[0]

      target_pos, target_sz = None, None
      for box, label, score in zip(output['boxes'], output['labels'], output['scores']):
        # search for a person with atleast 70% probability
        if score > 0.7 and label == 1:
          box = box.numpy()
          x, y = int(box[0]), int(box[1])
          w, h = int(box[2]) - x, int(box[3]) - y
          target_pos = np.array([x + w / 2, y + h / 2])
          target_sz = np.array([w, h])
          break
      assert target_pos is not None, "no person found on the first frame!"
      print("person found:", target_pos, target_sz)
      
      # init tracker
      state = siamese_init(frame, target_pos, target_sz, siammask, cfg['hp'], device=device)
    else:
      # track
      state = siamese_track(state, frame, mask_enable=True, refine_enable=True, device=device)
      location = state['ploygon'].flatten()
      mask = state['mask'] > state['p'].seg_thr
      
      frame[:, :, 2] = (mask > 0) * 255 + (mask == 0) * frame[:, :, 2]
      cv2.polylines(frame, [np.int0(location).reshape((-1, 1, 2))], True, (0, 255, 0), 3)
      
      video_writer.write(frame)
  
    f += 1
    # only on first 100 frames
    if f > 100:
      break
      
  video_capture.release()
  video_writer.release()
  
  # convert AVI to MP4
  !ffmpeg -y -loglevel info -i output.avi output.mp4
else:
  print("can't open the given input video file!")

Finally, visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

In [ ]:
! git clone https://github.com/burnpiro/farm-animal-tracking.git

Cloning into 'farm-animal-tracking'...
remote: Enumerating objects: 715, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 715 (delta 42), reused 50 (delta 21), pack-reused 621
Receiving objects: 100% (715/715), 29.04 MiB | 21.77 MiB/s, done.
Resolving deltas: 100% (412/412), done.


In [ ]:
! git clone https://github.com/tensorflow/models/tree/archive/research/object_detection.git

Cloning into 'object_detection'...
fatal: repository 'https://github.com/tensorflow/models/tree/archive/research/object_detection/' not found


In [ ]:
!cd /content/farm-animal-tracking && ls

assets			       helpers		   show_tracking.py
calculate_interval_score.py    LICENSE		   test_evaluation.py
crop_images		       model		   test.py
data			       output		   test_siamese.py
deepsort_tracker	       Readme.md	   test_video.py
eda			       requirements.txt    train_classificator.py
experiments		       run_detection.py    train_siamese.py
generate_siamese_emb_space.py  show_prediction.py


In [ ]:
!ls 

adc.json  farm-animal-tracking	sample_data  train


In [ ]:
!cd /content/farm-animal-tracking && pip install -r requirements.txt

In [ ]:
! python show_prediction.py

python3: can't open file 'show_prediction.py': [Errno 2] No such file or directory


In [ ]:
!cd /content/farm-animal-tracking && python show_tracking.py --video /content/train/Movie_10.mkv

Traceback (most recent call last):
  File "show_tracking.py", line 12, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


In [ ]:
!

In [ ]:
%%capture
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
search_id = "1OejSwght_F4tKbaLjgmB1srBnIJDNuU-"
search = drive.CreateFile({'id': search_id})
search.GetContentFile('train.zip')

In [ ]:
!unzip ./train.zip
!rm -r ./train.zip

Archive:  ./train.zip
   creating: train/
  inflating: train/Movie_5.mkv       
  inflating: train/Movie_7.mkv       
  inflating: train/Movie_6.mkv       
  inflating: train/Movie_10.mkv      
  inflating: train/Movie_1.mkv       
  inflating: train/Movie_3.mkv       
  inflating: train/Movie_2.mkv       
  inflating: train/Movie_4.mkv       
  inflating: train/Movie_8.mkv       
  inflating: train/Movie_9.mkv       
